PyTorch command line training script recipe for training state-of-the-art computer vision models with 8 GPUs. Source: PyTorch blog.
What we're going to cover

### what we are going to cover
The main concept of this section is: turn useful notebook code cells into reusable Python files.

Doing this will save us writing the same code over and over again.

There are two notebooks for this section:

    1. `05. Going Modular: Part 1 (cell mode)` - this notebook is run as a traditional Jupyter Notebook/Google Colab notebook and is a condensed version of notebook 04.
    2. `05. Going Modular: Part 2 (script mode)` - this notebook is the same as number 1 but with added functionality to turn each of the major sections into Python scripts, such as, data_setup.py and train.py.

The text in this document focuses on the code cells 05. Going Modular: Part 2 (script mode), the ones with `%%writefile ...` at the top.

In [1]:
import requests
import os
import zipfile
from pathlib import Path

# set path to a data folder 
data_path = Path('data/')
image_path = data_path /'pizza_steak_sushi'

# if the image folder doesn't exist download it and prepare it....\
if image_path.is_dir():
    print(f'{image_path} direcctory already exists ..... skippping dowload ')
else :
    print(f'{image_path} does not exist , creating onr ......')
    image_path.mkdir(parents = True,exist_ok = True)

# dowload pizza , steak and sushi data
with open(data_path /'pizza_steak_sushi.zip','wb') as f:
    request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
    print('downloading')
    f.write(request.content)
# unzip 
with zipfile.ZipFile(data_path/'pizza_steak_sushi.zip','r') as zip_ref:
    print('inzipping pizza , steak and sushi data...')
    zip_ref.extractall(image_path)

# remove zip file 
os.remove(data_path/'pizza_steak_sushi.zip')

data/pizza_steak_sushi direcctory already exists ..... skippping dowload 
downloading
inzipping pizza , steak and sushi data...


In [2]:
### set up trainig and testing paths 
train_dir = image_path/'train'
test_dir =image_path/'test'

In [3]:
## Create dataset and data loader 
import torch
from torch.utils.data import DataLoader
from torchvision import datasets , transforms
# write a trasform for image
data_trasform = transforms.Compose([
    # resize our image
    transforms.Resize(size =(64,64)),
    # filp the image randomly on the horizontal 
    transforms.RandomVerticalFlip(p =0.5),
    # turn the image into a torch.tensor
    transforms.ToTensor()
])
train_data = datasets.ImageFolder(root= train_dir,
                                 transform= data_trasform, # a transform for the data
                                 target_transform= None) # transform for the labels 

test_data = datasets.ImageFolder(root = test_dir ,
                                transform = data_trasform)

train_data , test_data

/home/anubhav/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


(Dataset ImageFolder
     Number of datapoints: 225
     Root location: data/pizza_steak_sushi/train
     StandardTransform
 Transform: Compose(
                Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
                RandomVerticalFlip(p=0.5)
                ToTensor()
            ),
 Dataset ImageFolder
     Number of datapoints: 75
     Root location: data/pizza_steak_sushi/test
     StandardTransform
 Transform: Compose(
                Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
                RandomVerticalFlip(p=0.5)
                ToTensor()
            ))

In [4]:
class_names = train_data.classes
class_names


['pizza', 'steak', 'sushi']

In [5]:
# get class names as dict
class_dict = train_data.class_to_idx
class_dict

{'pizza': 0, 'steak': 1, 'sushi': 2}

In [6]:
# check the lenghts of our datasets 
len(train_data) , len(test_data)

(225, 75)

In [7]:
# turn train and test datasets into dataloader's 
from torch.utils.data import DataLoader
BATCH_SIZE = 1
train_dataloader = DataLoader (dataset =train_data ,
                               batch_size =BATCH_SIZE ,
                              num_workers= 10,
                              shuffle= True)
test_dataloader = DataLoader (dataset =test_data ,
                              batch_size =BATCH_SIZE ,
                              num_workers= 10,
                              shuffle= False)

In [8]:
img ,lable =next(iter(train_dataloader))
print(f'Image shape:{img.shape}-> [batchsieze ,color_channles , height ,width]')
print(f'Lable {lable.shape}')

Image shape:torch.Size([1, 3, 64, 64])-> [batchsieze ,color_channles , height ,width]
Lable torch.Size([1])


### 2.1 Create Dataset and Dataloader in the script mode
Let's use the JUpyter magic function `.py`  file for creating dataloaders
We csn save a code cell's contents to a afile using %%writefile 


In [9]:
## create a directory for going modular
import os 
# os.mkdir('going_modular')

In [10]:
%%writefile going_modular/data_setup.py

"""
Cotntains functionlity for creating Pytorch DataLOader for imge classification data """
## Create dataset and data loader 

from torchvision import datasets , transforms
from torch.utils.data import DataLoader
import os

NUM_WORKERS =os.cpu_count()

def create_dataloaders(
    train_dir :str,
    test_dir:str,
    transform : transforms.Compose,
    batch_size = int,
    num_workers: int = NUM_WORKERS
    ):
    """
    Create training and testing DataLoader 
    Takes in a training directory and testing directory and turn them into Pytorch
    Dataset and then into PytorchLoader
    
    Args:
     train_dir  :Path to traning directory 
     test_dir : Path to testing directory .
     transform : trochvision transforms to traning and testing data.
     batch_size :NUmber of samples per batch in each of the DataLoader
     num_workers : An integers for number of wrokers per DataLoader

    Returns:
     A tuple of (traning_dataloader,test_dataloader,class_names ).
     Where class_names is a list of the target classes.
     Example usages  :
     train_dataloader ,test_dataloader ,class_naem = create_dataloaders(train_dir=path/to/train_dir),
     transform = some_transforms,
     batcch_size =32 
     num_workers =4
    """
    # use image folder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir,transform = transform)
    test_data  = datasets.ImageFolder(test_dir,transform= transform)

    # return class_name
    class_names = train_data.classes

    # turn image into dataloaders 
    train_dataloader = DataLoader(
       train_data,
        batch_size= batch_size,
        shuffle= True,
        num_workers= num_workers,
        pin_memory= True
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size= batch_size,
        shuffle= False,
        num_workers= num_workers,
        pin_memory= True
    )
    
    return train_dataloader,test_dataloader,class_names
    


Overwriting going_modular/data_setup.py


In [11]:
from going_modular import data_setup
train_dataloader,test_dataloader,class_names  = data_setup.create_dataloaders(train_dir=train_dir,test_dir=test_dir,transform=data_trasform,batch_size= 32)

In [14]:
train_dataloader ,test_dataloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x7fdc31e4f340>,
 ['pizza', 'steak', 'sushi'])

### Creating a model to script 

In [25]:
from torch import nn
class TinyVGG(nn.Module):
    """ 
    model architecture copying TInyVGG from CNN Expliainer 
    """
    def __init__(self,input_shape :int,
                hidden_units : int,
                output_shape : int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels= input_shape,
                     out_channels= hidden_units,
                     kernel_size= 3,
                     stride= 1,
                     padding = 0),
            nn.ReLU(),
            nn.Conv2d(in_channels= hidden_units,
                     out_channels= hidden_units,
                     kernel_size= 3,
                     stride = 1, 
                     padding = 0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, 
                        stride= 2) # default the stride size is same as the kernel size 
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels= hidden_units,
                     out_channels= hidden_units,
                     kernel_size= 3,
                     stride= 1,
                     padding = 0),
            nn.ReLU(),
            nn.Conv2d(in_channels= hidden_units,
                     out_channels= hidden_units,
                     kernel_size= 3,
                     stride = 1, 
                     padding = 0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, 
                        stride= 2) # default the stride size is same as the kernel size 
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features= hidden_units*13*13,
                     out_features= output_shape)
        )
    def forward(self,x):
        x = self.conv_block_1(x)
        # print(x.shape)
        x =self.conv_block_2(x)
        # print(x.shape)
        x =self.classifier(x)
        # print(x.shape)
        return x
        # return self.classifier(self.conv_block_2(self.conv_block_1(x)))

In [27]:
torch.manual_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_0 = TinyVGG(input_shape= 3, # number of colour channels in out image data
                 hidden_units= 10,
                 output_shape= len(class_names)).to(device)

model_0

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [28]:
img_batch,label_batch =next(iter(train_dataloader))

img_single ,label_single = img_batch[0].unsqueeze(dim =0),label_batch[0]

print(f'single image hape :{img_single.shape}')

model_0.eval()
with torch.inference_mode():
    pred = model_0(img_single.to(device))

print(f'output logits:\n{pred}\n')
print(f'output prediction problities :\n{torch.softmax(pred,dim =1)}\n')
print(f'output lable :\n{torch.argmax(torch.softmax(pred,dim =1 ),dim =1)}\n')
print(f'output actual :\n{label_single}\n')



single image hape :torch.Size([1, 3, 64, 64])
output logits:
tensor([[ 0.0208, -0.0020,  0.0095]], device='cuda:0')

output prediction problities :
tensor([[0.3371, 0.3295, 0.3333]], device='cuda:0')

output lable :
tensor([0], device='cuda:0')

output actual :
0



In [31]:
%%writefile going_modular/model_builder.py
"""
Contains Pytorch model code to instactiate model from a CNN expliner website
"""
from torch import nn
class TinyVGG(nn.Module):
    """ 
    model architecture copying TInyVGG from CNN Expliainer 
    """
    def __init__(self,input_shape :int,
                hidden_units : int,
                output_shape : int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels= input_shape,
                     out_channels= hidden_units,
                     kernel_size= 3,
                     stride= 1,
                     padding = 0),
            nn.ReLU(),
            nn.Conv2d(in_channels= hidden_units,
                     out_channels= hidden_units,
                     kernel_size= 3,
                     stride = 1, 
                     padding = 0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, 
                        stride= 2) # default the stride size is same as the kernel size 
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels= hidden_units,
                     out_channels= hidden_units,
                     kernel_size= 3,
                     stride= 1,
                     padding = 0),
            nn.ReLU(),
            nn.Conv2d(in_channels= hidden_units,
                     out_channels= hidden_units,
                     kernel_size= 3,
                     stride = 1, 
                     padding = 0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, 
                        stride= 2) # default the stride size is same as the kernel size 
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features= hidden_units*13*13,
                     out_features= output_shape)
        )
    def forward(self,x):
        x = self.conv_block_1(x)
        # print(x.shape)
        x =self.conv_block_2(x)
        # print(x.shape)
        x =self.classifier(x)
        # print(x.shape)
        return x
        # return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Overwriting going_modular/model_builder.py


In [38]:
import torch
from going_modular import model_builder
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(42)
input_shape = 3
output_shape = 3
model_1 = model_builder.TinyVGG(input_shape= input_shape,
                                output_shape=len(class_names),
                                hidden_units= 10).to(device)

In [39]:
model_1

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [41]:
img_batch,label_batch =next(iter(train_dataloader))

img_single ,label_single = img_batch[0].unsqueeze(dim =0),label_batch[0]

print(f'single image hape :{img_single.shape}')

model_1.eval()
with torch.inference_mode():
    pred = model_1(img_single.to(device))

print(f'output logits:\n{pred}\n')
print(f'output prediction problities :\n{torch.softmax(pred,dim =1)}\n')
print(f'output lable :\n{torch.argmax(torch.softmax(pred,dim =1 ),dim =1)}\n')
print(f'output actual :\n{label_single}\n')



single image hape :torch.Size([1, 3, 64, 64])
output logits:
tensor([[ 0.0234, -0.0057,  0.0138]], device='cuda:0')

output prediction problities :
tensor([[0.3376, 0.3280, 0.3344]], device='cuda:0')

output lable :
tensor([0], device='cuda:0')

output actual :
2



#### 4.1 turn traning function into script


In [46]:
%%writefile going_modular/engine.py
# create a train_step ()
from typing import Tuple, Dict, List
from tqdm.auto import tqdm
import torch

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
        model: A PyTorch model to be trained.
        dataloader: A DataLoader instance for the model to be trained on.
        loss_fn: A PyTorch loss function to minimize.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
        A tuple of training loss and training accuracy metrics.
        In the form (train_loss, train_accuracy). For example:

        (0.1112, 0.8743)
    """
    # Put the model in train mode
    model.train()
    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0
    # Loop through data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to the target device
        X, y = X.to(device), y.to(device)
        # 1. Forward pass
        y_pred = model(X)
        # 2. Calculate the loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate accuracy metric
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to evaluation mode and then
    runs through all of the required testing steps (forward
    pass, loss calculation).

    Args:
        model: A PyTorch model to be tested.
        dataloader: A DataLoader instance for the model to be tested on.
        loss_fn: A PyTorch loss function to calculate loss.
        device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
        A tuple of test loss and test accuracy metrics.
        In the form (test_loss, test_accuracy). For example:

        (0.1112, 0.8743)
    """
    # Put model in eval mode
    model.eval()
    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference mode
    with torch.inference_mode():
        # Loop through dataloader batches
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            # Forward pass
            test_pred_logits = model(X)
            # Calculate the loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate the accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch model through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
        model: A PyTorch model to be trained and tested.
        train_dataloader: A DataLoader instance for the model to be trained on.
        test_dataloader: A DataLoader instance for the model to be tested on.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        loss_fn: A PyTorch loss function to calculate loss on both datasets.
        epochs: An integer indicating how many epochs to train for.
        device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
        A dictionary of training and testing loss as well as training and
        testing accuracy metrics. Each metric has a value in a list for 
        each epoch.
        In the form: {train_loss: [...],
                      train_acc: [...],
                      test_loss: [...],
                      test_acc: [...]} 
        For example if training for epochs=2: 
                     {train_loss: [2.0616, 1.0537],
                      train_acc: [0.3945, 0.3945],
                      test_loss: [1.2641, 1.5706],
                      test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
              }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        # Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results


Overwriting going_modular/engine.py


In [48]:
from going_modular import engine


###  Save the model (script mode)

In [49]:
%%writefile going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing going_modular/utils.py


In [52]:
from going_modular import utils
utils.save_model

<function going_modular.utils.save_model(model: torch.nn.modules.module.Module, target_dir: str, model_name: str)>

### Train, evaluate and save the model (train.py)(by using our scripts)


In [53]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")


Writing going_modular/train.py


In [55]:
!python3 going_modular/train.py

/home/anubhav/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
  0%|                                                     | 0/5 [00:00<?, ?it/s]/home/anubhav/.local/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/home/anubhav/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Trig